In [226]:
import pandas as pd
import numpy as np

### Preprocessing for all possible TABLES

In [267]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [269]:
len(df["ID_FINCA"].unique())

1231

In [228]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [229]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [230]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_516\3219566741.py:2: FutureWarning: ['SUPERFICIE', 'PRODUCCION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [231]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [232]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [233]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [234]:
df.loc[df["MODO"]=="1", "MODO"]  = 0
df.loc[df["MODO"]=="2", "MODO"]  = 1

In [235]:
df = df.drop(index=df[df["CAMPAÑA"] == "22"].index)

### 1 - Table without SUPERFICIE

In [185]:
df_temporal = df.drop(axis = 1, columns = ["SUPERFICIE"])

In [186]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)
encoder = ce.OrdinalEncoder(cols = ["CAMPAÑA"])
df_encoded = encoder.fit_transform(df_temporal)

In [169]:
df_temporal.to_csv("D_T_1.csv")
df_encoded.to_csv("D_T_1_encoded.csv")

### 2 - Table without CAMPAÑA and SUPERFICIE

In [187]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA", "SUPERFICIE"])

In [188]:
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

In [172]:
df_temporal.to_csv("D_T_2.csv")
df_encoded.to_csv("D_T_encoded_2.csv")

### 3 - Table without CAMPAÑA

In [237]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA"])

In [238]:
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

In [239]:
train = df_encoded[(df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]

In [242]:
X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

In [246]:
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler

normalizer = StandardScaler()
norm_X = normalizer.fit_transform(X)
norm_to_predict = normalizer.transform(to_predict)

In [247]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor


model = {"model_name": "SVC", "model": MLPRegressor(), "params": {'hidden_layer_sizes':[(100),(1000,1000,1000)],
                                                                  'learning_rate':["constant", "adaptive"],}}

grid_search = GridSearchCV(model["model"], model["params"], scoring="r2")
grid_search.fit(norm_X, y)

C:\Users\xavid\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


GridSearchCV(estimator=MLPRegressor(),
             param_grid={'hidden_layer_sizes': [100, (100, 100, 100)],
                         'learning_rate': ['constant', 'adaptive']},
             scoring='r2')

In [248]:
print(grid_search.best_params_)
df = pd.DataFrame(grid_search.cv_results_)
df

{'hidden_layer_sizes': (100, 100, 100), 'learning_rate': 'adaptive'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,9.451171,3.131344,0.005322,0.000376,100,constant,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.774989,0.799511,0.870512,0.590862,0.665164,0.740207,0.099641,4
1,9.308895,1.867798,0.004107,0.000587,100,adaptive,"{'hidden_layer_sizes': 100, 'learning_rate': '...",0.784408,0.759720,0.861517,0.754148,0.586562,0.749271,0.089934,3
2,8.366059,0.976483,0.005138,0.000371,"(100, 100, 100)",constant,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.891863,0.824670,0.908488,0.864096,0.617077,0.821239,0.105953,2
3,8.098244,1.237529,0.004801,0.000393,"(100, 100, 100)",adaptive,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.821207,0.866981,0.892721,0.871692,0.829611,0.856443,0.026912,1


In [251]:
modelMLPR = MLPRegressor(hidden_layer_sizes=(1000, 1000, 1000), learning_rate="adaptive")
modelMLPR.fit(norm_X, y)
predicted = modelMLPR.predict(norm_to_predict)
final_df = to_predict
final_df["SUPERFICIE"] = predicted

In [253]:
predicted

array([1.06726886, 1.06726886, 2.07793888, ..., 6.70734351, 6.66371718,
       2.44470395])

In [254]:
final_df = pd.concat([final_df, train])

In [255]:
final_df

,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,ID_FINCA_10,...,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,SUPERFICIE
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,22215.0,660.000000,0.00000,1.067269
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,22215.0,660.000000,0.00000,1.067269
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,20978.0,520.000000,0.00000,2.077939
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,40722.0,520.000000,0.00000,2.741506
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,14126.0,654.501718,9.19244,1.658436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,28160.1,700.000000,0.00000,3.680000
8522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,41310.0,700.000000,0.00000,4.250000
8523,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,45420.0,700.000000,0.00000,4.160000
8524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,56140.0,700.000000,0.00000,4.750000


In [262]:
final_df.to_csv("D_T_3.csv")
final_df.to_csv("D_T_encoded_3.csv")

In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601 entries, 0 to 9600
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   CAMPAÑA      9601 non-null   int64  
 1   ID_FINCA     9601 non-null   int64  
 2   ID_ZONA      9601 non-null   int64  
 3   ID_ESTACION  9601 non-null   int64  
 4   ALTITUD      9547 non-null   object 
 5   VARIEDAD     9601 non-null   int64  
 6   MODO         9601 non-null   int64  
 7   TIPO         9601 non-null   int64  
 8   COLOR        9601 non-null   int64  
 9   SUPERFICIE   9601 non-null   float64
 10  PRODUCCION   8526 non-null   float64
dtypes: float64(2), int64(8), object(1)
memory usage: 825.2+ KB


In [277]:
df.loc[

TypeError: bad operand type for unary ~: 'list'